In [5]:
import sacred 
import tensorflow as tf

import keras 
import keras_genomics
import numpy as np
import keras.layers as k1
import simdna

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.layers import Input
from keras.engine import Layer
from keras.models import Sequential 
from keras.engine.base_layer import InputSpec
from keras.models import Model
from keras.models import load_model
from sacred import Ingredient

In [ ]:
model_ingredient = Ingredient('model')

@model_ingredient.config 
def config(dataset): 
    # Number of convolutional layers to apply
    num_conv = 3
    
    #Filters 
    filters = 15
    
    #Kernel Size 
    kernel_size = 15 
    
    #Pool Size for Max Pooling 
    pool_size = 40 
    
    #Strides for Max Pooling 
    strides = 40 
    
    #Whether to use rc layers or normal layers 
    rev_comp = False 
    
    #Whether the model is siamese or not 
    siamese = False
    
    #Whether to use regular dropout 
    dropout = False 
    
    #Dropout Rate: used for all types of dropout
    dropout_rate = 0.2
    
    #Whether to use RevCompSpatialDropout1D 
    spatial_dropout = False 
    
    #Whether to use MCRCDropout 
    mc_dropout = False
    
    #Type of Pooling, options are: 'max', 'avg', 'weight_dist' 
    pooling = 'max'
    
    #Units in dense layer 
    units = 100
    
    #Whether or not to use a siamese model
    siamese = False
    
@model_ingredient.capture  
def model(
    x, num_conv, filters, kernel_size, pool_size, strides, dropout,
    dropout_rate, spatial_dropout, spatial_dropout_rate, mc_dropout, 
    mc_dropout_rate, pooling, units
): 
    model = keras.models.Sequential()
    #Convolutional layers and dropout
    for i in range(num_conv): 
        if not rev_comp: 
            model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                          input_shape=keras_train_batch_generator[0][0].shape[1:],
                          padding="same")) 
        else: 
            model.add(keras_genomics.layers.RevCompConv1D(filters=filters, 
                                                          kernel_size=kernel_size, 
                                                          input_shape=keras_train_batch_generator[0][0].shape[1:], 
                                                          padding="same"))
        x = k1.core.Activation("relu")
        
        #Don't apply dropout before the max pooling layer
        if i != num_conv - 1: 
            if dropout: 
                model.add(k1.Dropout(dropout_rate))
            elif spatial_dropout:
                model.add(RevCompSpatialDropout1D(dropout_rate)) 
            elif mc_dropout: 
                model.add(MCRCDropout(dropout_rate))

    #Only needed with rc model
    if rev_comp: 
        model.add(RevCompSumPool())
    
    #Pooling layers
    if pooling == 'max': 
        model.add(k1.pooling.MaxPooling1D(pool_size = pool_size,padding = "same", strides = strides))
    elif pooling == 'avg':
        model.add(k1.pooling.AveragePooling1D(pool_size = pool_size, padding = "same", strides = strides))
    elif pooling == 'weight_dist'
        #ADD THIS IN LATER 
    
    model.add(Flatten())
    
    #Fully-connected layers 
    model.add(keras_genomics.layers.core.Dense(units = units, activation = "relu"))
    model.add(keras_genomics.layers.core.Dense(units = 1))  
    
    return model